## Align utterance in Memor to open subtitle parallel bitext

In [65]:
import json
from lxml import etree
import lxml
from tqdm import tqdm
import pickle as pkl

#### Load MEMOR Dataset

In [2]:
# Load Memor Dataset
with open('memor/data.json') as f:
    data = json.load(f)

In [89]:
"""
1.Get utterances from a certain episode using episode_key
2.Split utterances into segments using puncutation: , . ?
"""
def get_segments(episode_key, all_data):
    # Gather utteranes the defined episode
    episode = []
    for x in all_data:
        if x.strip().split('_')[0]==episode_key:
            episode.extend(data[x]['sentences'])

    # Split utterances into segments
    temp = []
    for x in episode:
        temp.extend(x.strip().split('.'))
    episode = temp

    temp = []
    for x in episode:
        temp.extend(x.strip().split(','))
    episode = temp

    temp = []
    for x in episode:
        temp.extend(x.strip().split('?'))
    episode = temp

    temp = []
    for x in episode:
        temp.extend(x.strip().split('!'))
    episode = temp

    temp = []
    for x in episode:
        if len(x.strip().split(' ')) > 5:
            temp.append(x)
    episode = temp

    return episode

In [91]:
results = get_segments('S01E01', data)
print(len(results))

89


In [46]:
episode = []
# Collect memor data from the same episode
for item in data:
    if item.strip().split('_')[0]=='S01E02':
        episode.append(data[item])

In [54]:
# Split sentence into small segments based on , and use segments to search the open subtitle corpus
segments = []
for x in episode:
    utts = x['sentences']
    for utt in utts:
        for sub_utt in utt.strip().split(','):
            sub_utt = sub_utt.strip().split('.')
            for item in sub_utt:
                temp = item.strip()
                if len(temp.split(' ')) > 6:
                    print(temp)
                    segments.append(temp)
                else:
                    print('====', temp)
        # temp_1 = utt.strip().split(',')

        # for segment in temp:
        #     if len(segment.split(" "))<6:
        #         print('====', segment)
        #     else:
        #         print(segment)
        #         segments.append(segment)
    print('=='*50)

==== no
==== i haven't
==== 
==== get used to it
==== 
==== yeah
==== i probably won't
==== but
==== 
==== 
==== hey sheldon
==== 
==== hi
==== 
hey raj!  still not talking to me
==== huh?
==== don't take it personally
==== it's his pathology
==== he can't talk to women
==== 
==== he can't talk to attractive women
or in your case a cheesecake-scented goddess!
==== so
==== there's gonna be some furniture delivered?
==== oh no
==== let's assume that they can
==== lois lane is falling
accelerating at an initial rate of 32 feet per second per second
superman swoops down to save her by reaching out two arms of steel
==== miss lane
who is now travelling at approximately 120 miles per hour
==== hits them
and is immediately sliced into three equal pieces
==== 
==== are you listening to yourself
it is well established that superman's flight is a feat of strength
it is an extension of his ability to leap tall buildings
an ability he derives from earth's yellow sun
==== 
==== yeah
and you don't h

In [55]:
print(len(segments))

67


In [57]:
for x in segments:
    print(x)

hey raj!  still not talking to me
or in your case a cheesecake-scented goddess!
accelerating at an initial rate of 32 feet per second per second
superman swoops down to save her by reaching out two arms of steel
who is now travelling at approximately 120 miles per hour
and is immediately sliced into three equal pieces
it is well established that superman's flight is a feat of strength
it is an extension of his ability to leap tall buildings
an ability he derives from earth's yellow sun
and you don't have a problem with that
a combination of the moon's solar reflection and the energy storage capacity of kryptonian skin cells
a combination of the moon's solar reflection and the energy storage capacity of kryptonian skin cells
i'm just going to go wash up
i have 26 hundred comic books in there
i challenge you to find a single reference to kryptonian skin cells
i'm just going to go wash up
i have 26 hundred comic books in there
i challenge you to find a single reference to kryptonian skin 

#### Load Open Subtitle BiText Corpus

In [3]:
# Load Open Subtitle Bitext
xml_file = etree.parse('../open_subtitle/en-zh_cn.tmx')

In [36]:
# Collect subtitles
en_subtitle = []
zh_subtitle = []
root = xml_file.getroot()
count = 0
for node in tqdm(root.iter()):
    if node.tag=='seg':
        if count % 2 ==0:
            en_subtitle.append(node.text)
        else:
            zh_subtitle.append(node.text)
        count += 1

49845028it [00:35, 1398557.88it/s]


In [39]:
# Check the processed subtitles
print(len(en_subtitle))
print(len(zh_subtitle))
for i in range(10):
    print(en_subtitle[i])
    print(zh_subtitle[i])
    print('=='*50)
for i in range(10):
    print(en_subtitle[-i])
    print(zh_subtitle[-i])
    print('=='*50)

Ah, this is greasy. I want to eat kimchee.
好想要吃泡菜
Is Chae Yoon's coordinator in here?
崔允的造型师在吗
Excuse me, aren't you Chae Yoon's coordinator?
请问一下 你是不是崔允的造型师
Yes. Me?
我吗
-Chae Yoon is done singing.
崔允已经唱完了
This lady right next to me... everyone knows who she is right?
这位小姐是谁 我想大家都知道吧
She is my forever idol...she is my rival whom I can never win against, she is the one who always makes me happy, and she is my mother.
她是我永远的偶像 也是我无法击倒的对手 她是令我怦然心动的恋人
Everyone should remember my mom as one of the greatest movie stars, but instead of becoming an actress, she wanted to become a singer just like me.
也许在各位的记忆里 我母亲 她是曾经红极一时的女演员
When I was stressed and exhausted, she'd always sing me a song while having me on her lap.
而是像我一样当歌手 当我感到累还有疲倦的时候 每一次她都让我
Today, I would like to share that moment with everyone.
然后为我唱一首歌 今天我想要 特别和各位
Ah, this is greasy. I want to eat kimchee.
好想要吃泡菜
Subtitle translation by Brian Athey
字幕翻译：
MY LADIES BY SHIN MICHIMA
（我的女人们）
It's because... that's the way I want it.
这是我自己的

In [67]:
pkl.dump(en_subtitle, open('en_subtitles.pkl', 'wb'))
pkl.dump(zh_subtitle, open('zh_subtitles.pkl', 'wb'))

#### Align the MRMOR Corpus with the Open Subtitle Corpus

In [87]:
# Load Open Subtitle
with open('en_subtitles.pkl', 'rb') as f:
    en_subtitle = pkl.load(f)
with open('zh_subtitles.pkl', 'rb') as f:
    zh_subtitle = pkl.load(f)

In [92]:
# Load MEMOR Dataset
with open('memor/data.json') as f:
    data = json.load(f)

In [97]:
episode_keys = set()
for x in data:
    episode_keys.add(x.strip().split('_')[0])
print(len(episode_keys))

In [103]:
episode_indexs = {}
for item in tqdm(episode_keys):
    temp = []
    segments = get_segments(item, data)
    for segment in segments:
        for i, subtitle in enumerate(en_subtitle):
            if segment in subtitle:
                temp.append([i, en_subtitle[i], zh_subtitle[i]])
    episode_indexs[item] = temp
pkl.dump(episode_indexs, open('episode_indexs.pkl', 'wb'))

  1%|          | 2/204 [01:40<2:49:04, 50.22s/it]


KeyboardInterrupt: 

In [59]:
for segment in segments:
    for i, subtitle in enumerate(en_subtitle):
        if segment in subtitle:
            print("Index:", i)
            print("Target_en:", segment)
            print("Target_zh:", zh_subtitle[i])
            print("Context:")
            for b in range(3):
                print(en_subtitle[i+2-b], zh_subtitle[i+2-b])
            print('=='*50)

Index: 4236022
Target_en: and is immediately sliced into three equal pieces
Target_zh: 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分
Context:
In what space, sir? In what space? 哪还有时间 先生?
Unless Superman matches her speed and decelerates. 除非超人赶上她的速度并减速
Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces. 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分
Index: 4236085
Target_en: because the evidence suggests the coffee table's having a tiny garage sale
Target_zh: 现在 我只是假设这是沙发 因为证据显示 咖啡桌上正进行一场小型旧货出售
Context:
No. 没有想过
Did it occur to you that not everyone has the compulsive need to sort organize and label the entire world around them? 你有没想过 并不是每个人都觉得必须要... 归类 整理 并将一切事物都贴上标签?
I'm just inferring that this as a couch because the evidence suggests the coffee table's having a tiny garage sale. 现在 我只是假设这是沙发 因为证据显示 咖啡桌上正进行一场小型旧货出售
Index: 3498567
Target_en: i just want you to know that
Target_zh: 大卫 我只要你知道我要走了
Context:
i told you David, 

In [60]:
print(en_subtitle[4236022], zh_subtitle[4236022])

Miss Lane, who is now traveling at approximately 120 miles an hour, hits them and is immediately sliced into three equal pieces. 莱恩小姐此时大约速度在120英里/时 猛撞上超人的手臂后 她会马上被切成三等分


In [63]:
b = 300
for i in range(b):
    print(en_subtitle[4236022-b+i], zh_subtitle[4236022-b+i])
for i in range(b):
    print(en_subtitle[4236022+i], zh_subtitle[4236022+i])

- He had a brand scorched on his hand... －他胳膊上有烙印 －他说出了发送机的位置
He told my people where the transmitter is, and he gave the card with the 1st part of the code. 还交出了密码第一部分的卡片
The Norwegian guy wasn't so compliant, 在挪威遇到的人不善言谈
- but we only needed his portion of the code. - The brand! －但我们只需要他的数字 －烙印
Now only a trifle is left - the 3rd portion of the code. 现在我就缺一小部分 密码第三部分， 但是我不知道该去哪里找
- Me either. - You're wrong. －我也是 －你错了
I'll prove you you're wrong. Just come here. Alone. 我向你证明， 如果你到我这里来，一个人
Think about it. You guys don't know the transmitter's location 好好想想吧，我知道发送机在哪里， 而你们，不知道
- and the code's two parts. －我知道密码的前两部分，而你们...
- We know them. ... －我们也知道
- You're bluffing. - No. －你虚张声势 －没有
Tell me, ten. 你能说出来吗？
Eight-six-six. 8－6－6...
Now you. ... 现在你说
Seven-eight-four-tree. 7－8－4－3
Great, what now? 很好，接下来那？
We should become partners, for some time, to find the missing part of the code together. 呃，我们应该暂时结成同盟 找到我们还没有的那部分密码
- And what ten? －然后呢？
- Then I'll kill you, or you will kill me, time